In [1]:
import os
os.chdir("..")

In [2]:
import torch
import torch.nn as nn
from fox.yolov3.models import Darknet
from fox.yolov3.utils.parse_config import parse_model_cfg
from fox.yolov3.models import create_modules
from fox.utils import extract_weights_from_model
from torchsummary import summary

In [3]:
class Head(torch.nn.Module):
    def __init__(self, config):
        super(Head, self).__init__()
        module_defs = parse_model_cfg(config)
        self.module_list, self.routs = create_modules(module_defs, (416, 416), config)

    def forward(self, x):
        out = []
        orig_inp = x
        for i, module in enumerate(self.module_list):
            name = module.__class__.__name__
            if name == 'WeightedFeatureFusion':
                x = module(x, out, orig_inp)
            elif name == 'FeatureConcat':
                x = module(x, out)
            else:
                x = module(x)

            out.append(x if self.routs[i] else [])

        return x



In [4]:
head = Head("config/yolov3-head.cfg")

In [29]:
summary(head.cuda(), (512, 52, 52))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 512, 52, 52]       2,359,296
       BatchNorm2d-2          [-1, 512, 52, 52]           1,024
         LeakyReLU-3          [-1, 512, 52, 52]               0
            Conv2d-4          [-1, 256, 52, 52]         131,072
       BatchNorm2d-5          [-1, 256, 52, 52]             512
         LeakyReLU-6          [-1, 256, 52, 52]               0
            Conv2d-7          [-1, 512, 52, 52]       1,179,648
       BatchNorm2d-8          [-1, 512, 52, 52]           1,024
         LeakyReLU-9          [-1, 512, 52, 52]               0
WeightedFeatureFusion-10          [-1, 512, 52, 52]               0
           Conv2d-11          [-1, 256, 52, 52]         131,072
      BatchNorm2d-12          [-1, 256, 52, 52]             512
        LeakyReLU-13          [-1, 256, 52, 52]               0
           Conv2d-14          [-1, 

In [3]:
pretrained_model = Darknet("config/yolov3-orig.cfg").to("cuda")
pretrained_data = torch.load("weights/yolo.pt")
pretrained_model.load_state_dict(pretrained_data["model"])

Model Summary: 225 layers, 6.25895e+07 parameters, 6.25895e+07 gradients


<All keys matched successfully>

In [4]:
extract_weights_from_model(pretrained_model, 36, "weights/yolo-detector.pt")

In [5]:
model = Darknet("config/yolov3-spp-detector.cfg").to("cuda")
data = torch.load("weights/yolo-detector.pt")
model.load_state_dict(data['model'], strict=False)

Model Summary: 147 layers, 5.93877e+07 parameters, 5.93877e+07 gradients


<All keys matched successfully>

In [6]:
summary(model, [(256, 52, 52), (512, 52, 52)])

yay
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
WeightedFeatureFusion-1          [-1, 256, 52, 52]               0
            Conv2d-2          [-1, 512, 26, 26]       1,179,648
       BatchNorm2d-3          [-1, 512, 26, 26]           1,024
         LeakyReLU-4          [-1, 512, 26, 26]               0
            Conv2d-5          [-1, 256, 26, 26]         131,072
       BatchNorm2d-6          [-1, 256, 26, 26]             512
         LeakyReLU-7          [-1, 256, 26, 26]               0
            Conv2d-8          [-1, 512, 26, 26]       1,179,648
       BatchNorm2d-9          [-1, 512, 26, 26]           1,024
        LeakyReLU-10          [-1, 512, 26, 26]               0
WeightedFeatureFusion-11          [-1, 512, 26, 26]               0
           Conv2d-12          [-1, 256, 26, 26]         131,072
      BatchNorm2d-13          [-1, 256, 26, 26]             512
        LeakyReLU-14        